In [45]:
import dotenv
import os
from neo4j import GraphDatabase
import pandas as pd
import urllib.request, json 
from operator import itemgetter
import numpy as np
from collections.abc import Iterable
from itertools import repeat
import requests
import pyreadr

### Neo4J AuraDB Setup

In [46]:
load_status = dotenv.load_dotenv("Neo4j-fccfe306-Created-2024-09-24.txt")
if load_status is False:
    raise RuntimeError('Environment variables not loaded.')

URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    print("Connection established.")

Connection established.


In [47]:
def execute_transaction(queries: Iterable[str],
                        param_dicts: Iterable[dict[str, str]] = None):
    
    '''Executes (query, param_dict) pairs on the database.
    Transaction based -- All queries must be successful for changes to be committed.'''

    param_dicts = param_dicts or repeat({}, len(queries)) # default to empty dicts for all queries
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session(database="neo4j") as session:
            with session.begin_transaction() as tx:
                for query, param_dict in zip(queries, param_dicts):
                    tx.run(query, param_dict)
                tx.commit()

In [48]:
def execute_query(query: str,
                  param_dicts: Iterable[dict[str, str]] = None):
    
    '''Executes a given query with every param_dict in param_dicts.
    If no param_dicts are passed, query is executed once without any parameters.'''
    
    if not param_dicts:
        return execute_transaction((query,))
    execute_transaction(repeat(query, len(param_dicts)), param_dicts)

In [49]:
def execute_query_from_file(path: str,
                            param_dicts: Iterable[dict[str, str]] = None):
    
    '''Executes a given query file with every param_dict in param_dicts.
    If no param_dicts are passed, query is executed once without any parameters.'''
    
    path = os.path.join(path)
    with open(path) as queryfile:
        query = queryfile.read()
    execute_query(query, param_dicts)

In [50]:
def execute_queries_from_folder(folder: str):

    '''Executes all query files in folder. Queries should not require any parameters.'''
    
    queries = []
    for file in os.listdir(folder):
        path = os.path.join(folder, file)
        with open(path) as queryfile:
            queries.append(queryfile.read())
    execute_transaction(queries)

In [51]:
# Resetting database
execute_query_from_file('cypher/utility/reset_graph.cql')
execute_query_from_file('cypher/utility/reset_constraints.cql')

In [52]:
# execute_queries_from_folder('cypher/setup/nodes/Company')

### Region Nodes

In [53]:
df_m49 = pd.read_csv('data/UNSD_m49.csv', sep=';')

In [54]:
continents = df_m49[['Region Code', 'Region Name']]\
                    .dropna()\
                    .drop_duplicates()\
                    .rename(columns={
                        'Region Code': 'm49',
                        'Region Name': 'name'
                    })

In [55]:
subregions = df_m49[['Sub-region Code', 'Sub-region Name']]\
                    .dropna()\
                    .drop_duplicates()\
                    .rename(columns={
                        'Sub-region Code': 'm49',
                        'Sub-region Name': 'name'
                    })

In [56]:
itdregions = df_m49[['Intermediate Region Code', 'Intermediate Region Name']]\
                    .dropna()\
                    .drop_duplicates()\
                    .rename(columns={
                        'Intermediate Region Code': 'm49',
                        'Intermediate Region Name': 'name'
                    })

In [57]:
regions = pd.concat([continents, subregions, itdregions], ignore_index=True)\
            .astype({'m49': int})
regions.head(10)

,m49,name
0,2,Africa
1,19,Americas
2,142,Asia
3,150,Europe
4,9,Oceania
5,15,Northern Africa
6,202,Sub-Saharan Africa
7,419,Latin America and the Caribbean
8,21,Northern America
9,143,Central Asia


In [58]:
# Adding region constraints
execute_queries_from_folder('cypher/setup/nodes/Region')

In [59]:
execute_query_from_file('cypher/nodes/Region.cql', regions.to_dict('records'))

### Country Nodes

In [60]:
countries = df_m49[['ISO-alpha3 Code', 'ISO-alpha2 Code', 'Country or Area']]\
                    .dropna()\
                    .drop_duplicates()\
                    .rename(columns={
                        'ISO-alpha3 Code': 'iso3',
                        'ISO-alpha2 Code': 'iso2',
                        'Country or Area': 'name'
                    })
countries.head(10)

,iso3,iso2,name
0,DZA,DZ,Algeria
1,EGY,EG,Egypt
2,LBY,LY,Libya
3,MAR,MA,Morocco
4,SDN,SD,Sudan
5,TUN,TN,Tunisia
6,ESH,EH,Western Sahara
7,IOT,IO,British Indian Ocean Territory
8,BDI,BI,Burundi
9,COM,KM,Comoros


In [61]:
# Adding country constraints
execute_queries_from_folder('cypher/setup/nodes/Country')

In [62]:
execute_query_from_file('cypher/nodes/Country.cql', countries.to_dict('records'))

### Country IS_IN Region Relationship

In [63]:
country_continent = df_m49[['ISO-alpha3 Code', 'Region Code']]\
                            .dropna()\
                            .drop_duplicates()\
                            .rename(columns={
                                'ISO-alpha3 Code': 'iso3',
                                'Region Code': 'm49'
                            })

In [64]:
country_subregion = df_m49[['ISO-alpha3 Code', 'Sub-region Code']]\
                            .dropna()\
                            .drop_duplicates()\
                            .rename(columns={
                                'ISO-alpha3 Code': 'iso3',
                                'Sub-region Code': 'm49'
                            })

In [65]:
country_itdregion = df_m49[['ISO-alpha3 Code', 'Intermediate Region Code']]\
                            .dropna()\
                            .drop_duplicates()\
                            .rename(columns={
                                'ISO-alpha3 Code': 'iso3',
                                'Intermediate Region Code': 'm49'
                            })

In [66]:
country_region = pd.concat([country_continent, country_subregion, country_itdregion], ignore_index=True)
country_region.head(10)

,iso3,m49
0,DZA,2.0
1,EGY,2.0
2,LBY,2.0
3,MAR,2.0
4,SDN,2.0
5,TUN,2.0
6,ESH,2.0
7,IOT,2.0
8,BDI,2.0
9,COM,2.0


In [67]:
execute_query_from_file('cypher/edges/Country_IS_IN_Region.cql', country_region.to_dict('records'))

### Country Aliases Property

In [68]:
df_alias = pd.read_csv('data/country_aliases.csv')

In [69]:
def split_alias(row):
    '''
    Splits a row if Alias contains multiple aliases seperated by " or "
    '''
    if ' or ' in row['Alias']:
        values = row['Alias'].split(' or ')
        return pd.DataFrame({'iso3': [row['iso3']] * len(values), 'Alias': values})
    return pd.DataFrame({'iso3': [row['iso3']], 'Alias': [row['Alias']]})

In [70]:
aliases = pd.concat([split_alias(row) for _, row in df_alias.iterrows()],
                  ignore_index=True)\
        .dropna()\
        .drop_duplicates()\
        .rename(columns={'Alias': 'alias'})
aliases.head(10)

,iso3,alias
5,AFG,Afghanistan
6,AFG,Islamic Republic of Afghanistan
7,AFG,Da Afganistan Islami Jumhoryat
8,AFG,Jomhuriyyeh Eslamiyyeh Afganestan
9,ALB,Albania
10,ALB,Republic of Albania
11,ALB,Republika e Shqipërisë
12,ALB,Arnavutluk
13,ALB,Arbanon
14,DZA,Algeria


In [71]:
execute_query_from_file('cypher/nodes/Country_aliases.cql', aliases.to_dict('records'))

### Country Stats

In [72]:
def get_worldbank(indicator: str) -> pd.DataFrame:
    '''
    Get indicator data using worldbank API
    '''
    with urllib.request.urlopen(f"https://api.worldbank.org/v2/country/all/indicator/{indicator}?format=json&per_page=20000") as url:
        data = json.load(url)[1]
    ind = data[0]['indicator']['value']
    iso3 = map(itemgetter('countryiso3code'), data)
    year = map(itemgetter('date'), data)
    value = map(itemgetter('value'), data)
    return pd.DataFrame({
        'iso3': iso3,
        'year': year,
        ind: value
    }).replace('', np.nan)\
      .dropna()\
      .set_index(['iso3', 'year'])

In [73]:
population = get_worldbank('SP.POP.TOTL')

In [74]:
gdp = get_worldbank('NY.GDP.MKTP.CD')

In [75]:
pv = get_worldbank('PV.EST')

In [76]:
ctr = pd.read_excel('data/corp_tax_rate.xlsx')\
        .melt(id_vars='iso_3',
              value_vars=range(1980, 2024),
              var_name='year',
              value_name='corporate_tax_rate')\
        .rename(columns={'iso_3': 'iso3'})\
        .astype({'year': str})\
        .set_index(['iso3', 'year'])

In [77]:
stats = pd.concat([population, gdp, pv, ctr], axis=1).sort_index()\
          .reset_index()\
          .rename(columns={
              'Population, total': 'population',
              'GDP (current US$)': 'gdp',
              'Political Stability and Absence of Violence/Terrorism: Estimate': 'pv',
              'corporate_tax_rate': 'corporate_tax_rate'
          })
stats.head(10)

,iso3,year,population,gdp,pv,corporate_tax_rate
0,ABW,1960,54608.0,NaN,NaN,NaN
1,ABW,1961,55811.0,NaN,NaN,NaN
2,ABW,1962,56682.0,NaN,NaN,NaN
3,ABW,1963,57475.0,NaN,NaN,NaN
4,ABW,1964,58178.0,NaN,NaN,NaN
5,ABW,1965,58782.0,NaN,NaN,NaN
6,ABW,1966,59291.0,NaN,NaN,NaN
7,ABW,1967,59522.0,NaN,NaN,NaN
8,ABW,1968,59471.0,NaN,NaN,NaN
9,ABW,1969,59330.0,NaN,NaN,NaN


We use 2022 stats for now

In [78]:
execute_query_from_file('cypher/nodes/Country_stats.cql', stats[stats['year'] == '2022'].to_dict('records'))

### Sector Node


The data used in this analysis comes from [bautheac/GICS](https://github.com/bautheac/GICS) packages the Global Industry Classification Standards (GICS) dataset for consumption in R.  The GICS hierarchy begins with 11 sectors and is followed by 24 industry groups, 68 industries, and 157 sub-industries. 

In [79]:
url = 'https://github.com/bautheac/GICS/raw/0c2b0e4c0ca56a0e520301fd978fc095ed4fc328/data/standards.rda'
response = requests.get(url)

rda_file_path = './data/standards.rda'
with open(rda_file_path, 'wb') as file:
    file.write(response.content)

# Load the .rda file using pyreadr
result = pyreadr.read_r(rda_file_path)

print(result.keys())  

df = result[list(result.keys())[0]]  

# Save the DataFrame as a CSV file and remove the rda file
df.to_csv('./data/standards.csv', index=False)

os.remove(rda_file_path)

print("Data has been saved as standards.csv")


odict_keys(['standards'])
Data has been saved as standards.csv


In [80]:
# data wrangling for industry/sector

def wrangling(csv_path):
    df = pd.read_csv(csv_path)
    
    df = df.dropna()

    df = df.drop_duplicates()
    
    df = df.rename(columns={
        'sector id': 'sector_id',
        'sector name': 'sector_name',
        'industry group id': 'industry_group_id',
        'industry group name': 'industry_group_name',
        'industry id': 'industry_id',
        'industry name': 'industry_name',
        'subindustry id': 'subindustry_id',
        'subindustry name': 'subindustry_name',
        'description': 'primary_activity'
    })

    
    df['sector_id'] = df['sector_id'].astype('Int64')  
    df['industry_group_id'] = df['industry_group_id'].astype('Int64')
    df['industry_id'] = df['industry_id'].astype('Int64')
    df['subindustry_id'] = df['subindustry_id'].astype('Int64')

    df.reset_index(drop=True, inplace=True)
    df.index += 1

    return df

df_standards = wrangling("./data/standards.csv")

In [81]:
df_standards.head()

,sector_id,sector_name,industry_group_id,industry_group_name,industry_id,industry_name,subindustry_id,subindustry_name,primary_activity
1,10,Energy,1010,Energy,101010,Energy Equipment & Services,10101010,Oil & Gas Drilling,Drilling contractors or owners of drilling rig...
2,10,Energy,1010,Energy,101010,Energy Equipment & Services,10101020,Oil & Gas Equipment & Services,"Manufacturers of equipment, including drilling..."
3,10,Energy,1010,Energy,101020,"Oil, Gas & Consumable Fuels",10102010,Integrated Oil & Gas,Integrated oil companies engaged in the explor...
4,10,Energy,1010,Energy,101020,"Oil, Gas & Consumable Fuels",10102020,Oil & Gas Exploration & Production,Companies engaged in the exploration and produ...
5,10,Energy,1010,Energy,101020,"Oil, Gas & Consumable Fuels",10102030,Oil & Gas Refining & Marketing,Companies engaged in the refining and marketin...


In [82]:
sector = df_standards[['sector_id', 'sector_name']] \
        .drop_duplicates() \
        .rename(columns={
            'sector_id': 'gics',
            'sector_name': 'name'
        })

In [83]:
# Adding sector constraints
execute_queries_from_folder('cypher/setup/nodes/Sector')

In [84]:
execute_query_from_file('cypher/nodes/Sector.cql', sector.to_dict('records'))

### Industry Node

In [85]:
industry = df_standards[['subindustry_id', 'subindustry_name', 'primary_activity']] \
           .drop_duplicates() \
            .rename(columns={
                'subindustry_id': 'gics',
                'subindustry_name': 'name',
                'primary_activity': 'description'
            })

In [86]:
# Adding industry constraints
execute_queries_from_folder('cypher/setup/nodes/Industry')

In [87]:
execute_query_from_file('cypher/nodes/Industry.cql', industry.to_dict('records'))

### Industry PART_OF Sector Relationship

In [88]:
industry_sector = df_standards[['subindustry_id', 'sector_id']] \
                  .drop_duplicates() \
                  .rename(columns={
                      'subindustry_id': 'industry_gics',
                      'sector_id': 'sector_gics'
                  })

In [90]:
execute_query_from_file('cypher/edges/Industry_PART_OF_Sector.cql', industry_sector.to_dict('records'))

### Country Sector Relationships

In [160]:
sector_gdp = map(get_worldbank, ('NV.AGR.TOTL.ZS', 'NV.IND.TOTL.ZS', 'NV.IND.MANF.ZS', 'NV.SRV.TOTL.ZS'))
sector_gdp = pd.concat(sector_gdp, axis=1)
sector_gdp.columns = sector_gdp.columns.str.replace(', value added (% of GDP)', '')
sector_gdp = sector_gdp\
            .reset_index()\
            .melt(id_vars=['iso3', 'year'],
                  var_name='sector',
                  value_name='gdp_share')
sector_gdp.sort_values(['iso3', 'year', 'sector']).head(10)

,iso3,year,sector,gdp_share
2200,ABW,1995,"Agriculture, forestry, and fishing",0.505922
12923,ABW,1995,Industry (including construction),15.396362
23646,ABW,1995,Manufacturing,2.517766
34369,ABW,1995,Services,81.036379
2199,ABW,1996,"Agriculture, forestry, and fishing",0.485830
12922,ABW,1996,Industry (including construction),14.987449
23645,ABW,1996,Manufacturing,2.510121
34368,ABW,1996,Services,80.975304
2198,ABW,1997,"Agriculture, forestry, and fishing",0.429978
12921,ABW,1997,Industry (including construction),17.133844


In [161]:
sector_employment = map(get_worldbank, ('SL.AGR.EMPL.ZS', 'SL.IND.EMPL.ZS', 'SL.SRV.EMPL.ZS'))
sector_employment = pd.concat(sector_employment, axis=1)
sector_employment.columns = sector_employment.columns\
                              .str.replace(' (% of total employment) (modeled ILO estimate)', '')\
                              .str.replace('Employment in ', '')
sector_employment = sector_employment\
                    .reset_index()\
                    .melt(id_vars=['iso3', 'year'],
                          var_name='sector',
                          value_name='employment_share')
sector_employment.sort_values(['iso3', 'year', 'sector']).head(10)

,iso3,year,sector,employment_share
31,AFE,1991,agriculture,66.104830
7422,AFE,1991,industry,9.957919
14813,AFE,1991,services,23.937250
30,AFE,1992,agriculture,66.293264
7421,AFE,1992,industry,9.882672
14812,AFE,1992,services,23.824064
29,AFE,1993,agriculture,66.209034
7420,AFE,1993,industry,9.849627
14811,AFE,1993,services,23.941341
28,AFE,1994,agriculture,66.107798
